### Luis Fernando Jose Ruiz Ortega


# Gridworld y su solución como MDPs

En este trabajo definiremos el ambiente de Gridworld y su solución como un MDP.
Gridworld es un ambiente clásico de prueba dentro del aprendizaje por refuerzo. Durante este taller definiremos el modelo básico del ambiente, que extenderemos incrementalmente de acuerdo a las necesidades del algoritmo de solución.

## Ambiente 🌎

El ambiente de gridworld se define como una cuadricula de `nxm`. El ambiente tiene obstaculos, es decir casillas por las cuales no puede pasar el agente. Al chocar con un obstaculo, el agente terminaría en el mismo estado inicial. Además, el ambiente tiene una casilla de inicio, y algunas casillas de salida. Un ejemplo del ambiente para el caso `3x4` se muestra a continuación.

![gridworld.png](./img/gridworld.png)

En este ejemplo del ambiente el agente comienza en la casilla inferior izquierda y tiene como objetivo llegar a la casilla de salida verde, con recompensa 1. La otra casilla de salida, tiene recompensa -1.


### Task 1.

#### ¿Cómo podemos codificar el ambiente?

De una definición completa del ambiente, como una clase de python llamada `Environment`, estableciendo:

1. Un atributo que define la cuadrícula (`board`). El ambiente recibirá una matriz como parámetro describiendo la cuadrícula en el momento de su creación. Definiremos las casillas por las que puede pasar el agente como casillas vacias, las casillas por las que no puede pasar el agente con un valor none `#` y las casillas de salida con el valor asociado a la recompensa definidas para cada una de ellas.
2. Un atributo `nrows` para almacenar la cantidad de filas de la cuadrícula.
3. Un atributo `ncols` para almacenar la cantidad de columnas de la cuadrícula.
4. Un atributo `initial_state` para almacenar el estado inicial del agente dentro del ambiente.
5. Un atributo con el estado actual (`current_state`) en el que se encuentra el agente. El valor de `current_state` se definirá como una tupla
6. Un atributo `P` que guarda la matriz de probabilidades de cada una de las acciones para cada estado. Dicha matriz esta definida por parámetro.

Un ejemplo de la definición del tablero para el caso de 5x5 de la figura anterior se da a continuación.

```
board = [['', ' ', ' ',  '+1'],
         [' ', '#', ' ',  '-1'],
         ['S', ' ', ' ', ' ']]
```

En el ejemplo `S` denota el estado inicial y `'#'` la casilla prohibida (manejaremos esta convención para todos los ambientes de gridworld).

De forma similar a la definición del `board` la matriz de probabilidades `P` se define como:

```
P = [[[0.1, 0.1, 0, 0.8], [0.1, 0.1, 0, 0.8], [0.1, 0.1, 0, 0.8],  [1]],
         [[0.8, 0, 0.1, 0.1], '#', [0.8, 0, 0.1, 0.1],  [-1]],
         [[0.8, 0, 0.1, 0.1], [0.1, 0.1, 0.8, 0], [0.1, 0.1, 0.8, 0], [0.1, 0.1, 0.8, 0]]]
```

Para la definición de `P` vamos a entender cada una de las posiciones de la probabilidad en el orden (`'up', 'down', 'left', 'right'`). Adicionalmente, vamos a suponer que la casilla da la probabilidad de tal forma que el agente siempre toma la acción en dirección al objetivo (la acción que tiene probabilidad `0.8`). Por ejemplo, para la casilla superior izquierda la probabilidad de tomar la acción `right` y llegar a la casilla de arriba es de `0.1`, a la casilla de abajo con probabilidad `0.1` y a la casilla de la derecha con probabilidad `0.8` (el agente nunca puede llegar a la casilla de la izquierda). En las casillas de salida, el agente solo tiene una posibilidad que es tomar la acción `exit` que le da la recompensa asociada a la casilla al agente.

#### Comportamiento del ambiente

Una vez definido el ambiente definimos su comportamiento. Para ello requerimos los siguientes métodos:

1. `get_current_state` que no recibe parámetros y retorna el estado actual (la casilla donde se encuentra el agente)
2. `get_posible_actions` que recibe el estado actual del agente como parámetro y retorna las acciones disponibles para dicho estado. Las acciones estarán dadas por su nombre (`'up', 'down', 'left', 'right'`) para las casillas normales y (`'exit'`) para las casillas de salida. Como convención definiremas que el agente siempre puede moverse en todas las direcciones, donde un movimiento en dirección de un obstáculo o los límites del ambiente no tienen ningún efecto visible en la posición del agente.
3. `do_action` que recibe como parámetro la acción a ejecutar y retorna el valor de la recompensa y el nuevo estado del agente, como un pareja `reward, new_state`. Note que `do_action` esta restringida por la matriz de probabilidad `P` para la ejecución real de las acciones.
4. `reset` que no recibe parámetros y restablece el ambiente a su estado inicial.
5. `is_terminal` que no recibe parámetros y determina si el agente está en el estado final o no. En nuestro caso, el estado final estará determinado por las casillas de salida (i.e., con un valor definido).


In [ ]:
class Environment:
    
    board : list[list]
    nrows : int
    ncols : int
    initial_state : tuple
    current_state : tuple
    P : list[list[list[float]]]
    
    def __init__(self, board, P):
        self.board = board
        self.P = P
        self.nrows = len(board)
        self.ncols = len(board[0])
        self.initial_state = (0, 0) 
        self.current_state = (0, 0)
        
        for r in range(self.nrows):
            for c in range(self.ncols):
                if self.board[r][c] == 'S':
                    self.initial_state = (r, c)
                    self.current_state = (r, c)
                    break
    
    
    def get_current_state(self):
        return self.current_state
    
    def get_posible_actions(self, current_state):
        r, c = current_state
        probs = self.P[r][c]
        
        if len(probs) == 1 or probs == '#':
            return None
            
        actions_names = ['up', 'down', 'left', 'right']
        
        for i, prob in enumerate(probs):
            if prob == 0.8:
                return actions_names[i]
        
        return None
    
    def do_action(self, action):
        r, c = self.current_state
        
        if self.is_terminal(): 
            return self.board[self.current_state[0]][self.current_state[1]]

        dr, dc = 0, 0
        if action == 'up':    dr, dc = -1, 0
        elif action == 'down':  dr, dc = 1, 0
        elif action == 'left':  dr, dc = 0, -1
        elif action == 'right': dr, dc = 0, 1
        
        next_r = r + dr
        next_c = c + dc
        
        
        if not (0 <= next_r < self.nrows and 0 <= next_c < self.ncols):
            new_state = (r, c)
            reward = 0
            
        elif self.board[next_r][next_c] == '#':
            self.reset()
            
        else:
            new_state = (next_r, next_c)
            reward = 0
            
            casilla_destino = self.board[next_r][next_c]
            if casilla_destino not in ['', ' ', 'S']:
                reward = float(casilla_destino)

        self.current_state = new_state
        
        return reward, new_state
    
    def reset(self):
        self.current_state = self.initial_state
    
    def is_terminal(self):
        r, c = self.current_state
        casilla = self.board[r][c]
        
        no_terminales = ['', ' ', 'S', '#']
        
        if casilla not in no_terminales:
            return True 
        return False
    

In [325]:
# Ejemplo de tablero

board = [['', ' ', ' ',  '+1'],
         [' ', '#', ' ',  '-1'],
         ['S', ' ', ' ', ' ']]

P = [[[0.1, 0.1, 0, 0.8], [0.1, 0.1, 0, 0.8], [0.1, 0.1, 0, 0.8],  [1]],
     [[0.8, 0, 0.1, 0.1], '#', [0.8, 0, 0.1, 0.1],  [-1]],
     [[0.8, 0, 0.1, 0.1], [0.1, 0.1, 0.8, 0], [0.1, 0.1, 0.8, 0], [0.1, 0.1, 0.8, 0]]]

env = Environment(board, P)
print(f"Inicio en: {env.current_state}")


pasos = 0
while not env.is_terminal():
    
    accion_a_tomar = env.get_posible_actions(env.current_state)
    
    if accion_a_tomar is None:
        print("Error: No encontré acción de 0.8 o estoy en un lugar invalido")
        break
        
    estado_anterior = env.current_state
    reward, nuevo_estado = env.do_action(accion_a_tomar)
    
    print(f"Paso {pasos}: Estaba en {estado_anterior} -> Acción: '{accion_a_tomar}' -> Fui a {nuevo_estado}. Reward: {reward}")
    
    pasos += 1

if env.is_terminal():
    print(f"Llegué a un estado final con recompensa: {reward}")

Inicio en: (2, 0)
Paso 0: Estaba en (2, 0) -> Acción: 'up' -> Fui a (1, 0). Reward: 0
Paso 1: Estaba en (1, 0) -> Acción: 'up' -> Fui a (0, 0). Reward: 0
Paso 2: Estaba en (0, 0) -> Acción: 'right' -> Fui a (0, 1). Reward: 0
Paso 3: Estaba en (0, 1) -> Acción: 'right' -> Fui a (0, 2). Reward: 0
Paso 4: Estaba en (0, 2) -> Acción: 'right' -> Fui a (0, 3). Reward: 1.0
Llegué a un estado final con recompensa: 1.0


Teniendo en cuenta la definición del agente, genere un ambiente de `10x10` como se muestra a continuación.

![evaluacion.png](./img/evaluacion.png)


In [3]:
board_10x10 = [[' ' for _ in range(10)] for _ in range(10)]

board_10x10[0][0] = 'S'

for c in [1, 2, 3, 4, 6, 7, 8]:
    board_10x10[2][c] = '#'

for r in [3, 4, 5, 6, 7]:
    board_10x10[r][4] = '#'

board_10x10[4][5] = '-1' # R: -1
board_10x10[5][5] = '+1' # R: 1
board_10x10[7][5] = '-1' # R: -1
board_10x10[7][6] = '-1' # R: -1

print("Tablero 10x10 generado:")
for row in board_10x10:
    print(row)

Tablero 10x10 generado:
['S', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', '#', '#', '#', '#', ' ', '#', '#', '#', ' ']
[' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', '#', '-1', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', '#', '+1', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', '#', '-1', '-1', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']


### Task 2.

Plantee el problema de MDP para cada una de las casillas. Especifique el estado de inicio, las transiciones y su probabilidad (suponiendo que todas las acciones sucede con probabilidad de 0.25) y los estados de fin con su recompensa.
¿Cómo serían las recompensas esperadas para cada estado?


- Estados(S) = Son las 100 casillas, coordenadas (fila, columna) desde (0,0) hasta (9,9).

- Acciones = up, down left, rigth

- Probabilidades:
  - 0,25 para las 4 direcciones

- Recompensas:
  - +1 (5,5)
  - -1 (4,5), (7,5), (7,6)
  - 0 Demas casillas


In [327]:
import numpy as np

def calcular_recompensas_esperadas(board):
    rows = len(board)
    cols = len(board[0])
    
    expected_rewards = np.zeros((rows, cols))
    
    actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    for r in range(rows):
        for c in range(cols):
            if board[r][c] in ['#', '+1', '-1']:
                continue
            
            total_reward = 0
            
            for dr, dc in actions:
                nr, nc = r + dr, c + dc
                
                r_val = 0
                
                if not (0 <= nr < rows and 0 <= nc < cols):
                    r_val = 0
                
                elif board[nr][nc] == '#':
                    r_val = 0 
                
                elif board[nr][nc] in ['+1', '-1']:
                    r_val = float(board[nr][nc])
                
                else:
                    r_val = 0
                
                total_reward += r_val
            
            expected_rewards[r][c] = total_reward * 0.25
            
    return expected_rewards


matriz_esperada = calcular_recompensas_esperadas(board_10x10)

print("--- Matriz de Recompensas Esperadas ---")
for r in range(10):
    fila_str = ""
    for c in range(10):
        val = matriz_esperada[r][c]
        if board_10x10[r][c] == '#':
            fila_str += "  #   "
        elif val == 0:
            fila_str += "  .   "
        else:
            fila_str += f"{val:+.2f} "
    print(fila_str)

--- Matriz de Recompensas Esperadas ---
  .     .     .     .     .     .     .     .     .     .   
  .     .     .     .     .     .     .     .     .     .   
  .     #     #     #     #     .     #     #     #     .   
  .     .     .     .     #   -0.25   .     .     .     .   
  .     .     .     .     #     .   -0.25   .     .     .   
  .     .     .     .     #     .   +0.25   .     .     .   
  .     .     .     .     #     .   -0.25   .     .     .   
  .     .     .     .     #     .     .   -0.25   .     .   
  .     .     .     .     .   -0.25 -0.25   .     .     .   
  .     .     .     .     .     .     .     .     .     .   


### Task 3.

Bajo la definción del problema anterior, suponga que cada acción tiene una probabilidad de éxito de 60%, con probabilidad de 20% se ejecutará la sigiente acción (en dirección de las manesillas del reloj), con probabilidad de 10% se ejecutará la sigiente acción (en contra de las manesillas del reloj) y con probabilidad de 10% no pasará nada. Bajo estas condiciones, ¿Cómo serían las recompensas esperadas para cada estado?

Codifique el ambiente para el gridworld de `10x10` utilizando esta función de probabilidad. En esta codificación del ambiente no es necesario pasar la matriz `P` como parámetro, pero esta información se debe tener en cuenta en la función `do_action`.

Tenga en cuenta que la calidad del programa que entreguen será tenida en cuentra dentro de la calificación.


In [328]:
import numpy as np

def calcular_recompensas(board):
    rows = len(board)
    cols = len(board[0])
    
    expected_rewards = np.zeros((rows, cols))
    
    moves = {
        'up':    (-1, 0),
        'right': (0, 1),
        'down':  (1, 0),
        'left':  (0, -1)
    }
    ordered_actions = ['up', 'right', 'down', 'left']
    
    prob_success = 0.6
    prob_cw      = 0.2  
    prob_ccw     = 0.1 
    prob_stay    = 0.1
    
    for r in range(rows):
        for c in range(cols):
            if board[r][c] in ['#', '+1', '-1']:
                continue
            
            sum_expected_action_rewards = 0
            
            for intended_action in ordered_actions:
                idx = ordered_actions.index(intended_action)
                
                outcomes = [
                    (ordered_actions[idx], prob_success),          
                    (ordered_actions[(idx + 1) % 4], prob_cw),      
                    (ordered_actions[(idx - 1) % 4], prob_ccw),     
                    ('stay', prob_stay)                            
                ]
                
                action_reward = 0
                
                for real_move_name, prob in outcomes:
                    dr, dc = 0, 0
                    if real_move_name != 'stay':
                        dr, dc = moves[real_move_name]
                    
                    nr, nc = r + dr, c + dc
                    
                    val_reward = 0
                    if not (0 <= nr < rows and 0 <= nc < cols):
                        val_reward = 0
                    elif board[nr][nc] == '#':
                        val_reward = 0 
                    else:
                        contenido = board[nr][nc]
                        if contenido in ['+1', '-1']:
                            val_reward = float(contenido)
                        else:
                            val_reward = 0
                            
                    action_reward += prob * val_reward
                
                sum_expected_action_rewards += action_reward
            
            expected_rewards[r][c] = sum_expected_action_rewards * 0.25
            
    return expected_rewards

matriz = calcular_recompensas(board_10x10)

print("--- Matriz de Recompensas Esperadas---")
for r in range(10):
    fila_str = ""
    for c in range(10):
        val = matriz[r][c]
        if board_10x10[r][c] == '#':
            fila_str += "  #   "
        elif val == 0:
            fila_str += "  .   "
        else:
            fila_str += f"{val:+.3f} "
    print(fila_str)

--- Matriz de Recompensas Esperadas---
  .     .     .     .     .     .     .     .     .     .   
  .     .     .     .     .     .     .     .     .     .   
  .     #     #     #     #     .     #     #     #     .   
  .     .     .     .     #   -0.225   .     .     .     .   
  .     .     .     .     #     .   -0.225   .     .     .   
  .     .     .     .     #     .   +0.225   .     .     .   
  .     .     .     .     #   +0.000 -0.225   .     .     .   
  .     .     .     .     #     .     .   -0.225   .     .   
  .     .     .     .     .   -0.225 -0.225   .     .     .   
  .     .     .     .     .     .     .     .     .     .   


## Simulacion task3


In [ ]:
import random
import time

class EnvironmentTask3:
    def __init__(self, board):
        self.board = board
        self.nrows = len(board)
        self.ncols = len(board[0])
        self.initial_state = (0, 0)
        self.current_state = (0, 0)
        
        for r in range(self.nrows):
            for c in range(self.ncols):
                if self.board[r][c] == 'S':
                    self.initial_state = (r, c)
                    self.current_state = (r, c)
                    break

    def is_terminal(self):
        r, c = self.current_state
        return self.board[r][c] not in ['', ' ', 'S', '#']

    def do_action(self, intended_action):
        """
        Lógica:
        - 60% Intento original
        - 20% Clockwise (90° derecha)
        - 10% Counter-Clockwise (90° izquierda)
        - 10% Stay (Quieto)
        """
        directions = ['up', 'right', 'down', 'left']
        
        if intended_action not in directions:
            return 0, self.current_state

        idx = directions.index(intended_action)
        
        act_success = directions[idx]             
        act_cw      = directions[(idx + 1) % 4]   
        act_ccw     = directions[(idx - 1) % 4]   
        act_stay    = 'stay'
        
        outcomes = [act_success, act_cw, act_ccw, act_stay]
        probs    = [0.6,         0.2,    0.1,     0.1]
        
        real_move = random.choices(outcomes, weights=probs, k=1)[0]
        
        r, c = self.current_state
        dr, dc = 0, 0
        
        if real_move == 'up':    dr, dc = -1, 0
        elif real_move == 'down':  dr, dc = 1, 0
        elif real_move == 'left':  dr, dc = 0, -1
        elif real_move == 'right': dr, dc = 0, 1
        
        next_r, next_c = r + dr, c + dc
        
        reward = 0
        final_state = (r, c) 
        
        if 0 <= next_r < self.nrows and 0 <= next_c < self.ncols:
            contenido = self.board[next_r][next_c]
            
            if contenido == '#':
                final_state = self.initial_state
            else:
                final_state = (next_r, next_c)
                
                if contenido not in ['', ' ', 'S']:
                    reward = float(contenido)

        self.current_state = final_state
        return reward, final_state, real_move


In [339]:
board_10x10 = [[' ' for _ in range(10)] for _ in range(10)]
board_10x10[0][0] = 'S'
for c in [1, 2, 3, 4, 6, 7, 8]: board_10x10[2][c] = '#'
for r in [3, 4, 5, 6, 7]: board_10x10[r][4] = '#'
board_10x10[4][5] = '-1' 
board_10x10[5][5] = '+1' 
board_10x10[7][5] = '-1' 
board_10x10[7][6] = '-1'

env3 = EnvironmentTask3(board_10x10)


print(f"Inicio en: {env3.current_state}\n")

pasos = 0
max_pasos = 1000

while not env3.is_terminal() and pasos < max_pasos:
    
    intento = random.choice(['up', 'down', 'left', 'right'])
    
    recompensa, nuevo_estado, movimiento_real = env3.do_action(intento)
    
    info_extra = ""
    if intento != movimiento_real and movimiento_real != 'stay':
        info_extra = "Cambio"
    elif movimiento_real == 'stay':
        info_extra = "(Se quedó quieto)"
    elif nuevo_estado == env3.initial_state and movimiento_real != 'stay':
        info_extra = "Choque"
        
    print(f"Paso {pasos}: Intentó '{intento.upper()}' -> Salió '{movimiento_real}' {info_extra}")
    print(f"        Posición: {nuevo_estado}, Reward: {recompensa}")
    print("-" * 30)
    
    pasos += 1

if env3.is_terminal():
    print(f"\n LLego a una casilla terminal con Reward: {recompensa}")
else:
    print(f"\nSe detuvo por límite de pasos ({max_pasos}).")

Inicio en: (0, 0)

Paso 0: Intentó 'UP' -> Salió 'up' Choque
        Posición: (0, 0), Reward: 0
------------------------------
Paso 1: Intentó 'UP' -> Salió 'up' Choque
        Posición: (0, 0), Reward: 0
------------------------------
Paso 2: Intentó 'RIGHT' -> Salió 'right' 
        Posición: (0, 1), Reward: 0
------------------------------
Paso 3: Intentó 'DOWN' -> Salió 'stay' (Se quedó quieto)
        Posición: (0, 1), Reward: 0
------------------------------
Paso 4: Intentó 'DOWN' -> Salió 'down' 
        Posición: (1, 1), Reward: 0
------------------------------
Paso 5: Intentó 'LEFT' -> Salió 'up' Cambio
        Posición: (0, 1), Reward: 0
------------------------------
Paso 6: Intentó 'UP' -> Salió 'right' Cambio
        Posición: (0, 2), Reward: 0
------------------------------
Paso 7: Intentó 'UP' -> Salió 'up' 
        Posición: (0, 2), Reward: 0
------------------------------
Paso 8: Intentó 'DOWN' -> Salió 'down' 
        Posición: (1, 2), Reward: 0
---------------------

### Task 4.

Defina una situación de la vide real (de su escogencia) como un MDP.


Situacion : Un jugador de baloncesto practica lanzamientos de 3 puntos. El objetivo es maximizar la cantidad de puntos anotados en una sesión. Se asume que el jugador tiene estados de "confianza" que afectan su precisión.


Estados (S):

- Neutro : Estado inicial o despues de fallar
- Racha : Después de encestar. Decimos que la confianza aumenta la precisión.
- Frio : Después de fallar 2 veces seguidas.
- Fin : Termina la practica


Acciones (A):

- Tiro Estándar: Un tiro con mecánica controlada y equilibrada.

- Tiro Rápido: Un tiro más rapido. Permite sacar más tiros en menos tiempo, pero con menor precisión.


Probabilidades (P):

| Tiro estandar | Estado | Neutro | Racha | Frio |
| ------------- | ------ | ------ | ----- | ---- |
| ----          | Neutro | 0,4    | 0,4   | 0,2  |
| ----          | Racha  | 0,4    | 0,6   | 0    |
| ----          | Frio   | 0,3    | 0     | 0,7  |


| Tiro rapido | Estado | Neutro | Racha | Frio |
| ----------- | ------ | ------ | ----- | ---- |
| ----        | Neutro | 0,3    | 0,3   | 0,4  |
| ----        | Racha  | 0,5    | 0,5   | 0    |
| ----        | Frio   | 0,2    | 0     | 0,8  |


Recompensas (R):

- +3 por cada enceste (éxito).

- -1 por cada fallo.

- +1 extra si se encesta usando Tiro Rápido (por la satisfacción de meter un tiro difícil), siendo entonces un total de +4.
